# 🏪 서울시 상권유형 분석 - EDA
- **데이터**: 서울시 상권분석서비스(영역-상권)
- **목적**: 상권유형(골목/발달/전통시장/관광특구) 데이터가 회귀분석에 적합한지 확인

## 1. 데이터 로드

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정 (안티그래비티용)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 로드 완료!")

In [ ]:
# 상권영역 데이터 로드
df = pd.read_csv('data/district_type.csv', encoding='cp949')
print(f"✅ 데이터 로드 완료! 총 {len(df):,}개 row")

## 2. 전체 데이터 개요

In [ ]:
print("📊 전체 데이터 개요")
print("=" * 50)
print(f"전체 row 수: {len(df):,}개")
print(f"컬럼 수: {len(df.columns)}개")
print(f"상권유형 수: {df['상권_구분_코드_명'].nunique()}개")
print(f"자치구 수: {df['자치구_코드'].nunique()}개")
print(f"행정동 수: {df['행정동_코드'].nunique()}개")

In [ ]:
# 컬럼 확인
print("📋 컬럼 목록:")
print(df.columns.tolist())

In [ ]:
# 데이터 샘플
print("📋 데이터 샘플:")
df.head()

## 3. 상권유형별 분포

In [ ]:
print("🏪 상권유형별 개수")
print("=" * 50)
type_counts = df['상권_구분_코드_명'].value_counts()
print(type_counts)
print()
print("비율:")
print((type_counts / len(df) * 100).round(1).astype(str) + '%')

In [ ]:
# 상권유형별 파이차트
plt.figure(figsize=(8, 8))
colors = ['#66b3ff', '#99ff99', '#ffcc99', '#ff9999']
plt.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('서울시 상권유형별 분포', fontsize=14)
plt.tight_layout()
plt.show()

## 4. 행정동별 상권유형 집계

In [ ]:
# 행정동별 상권유형 개수 피벗
dong_type = df.groupby(['행정동_코드', '행정동_코드_명', '상권_구분_코드_명']).size().unstack(fill_value=0)
dong_type['총_상권수'] = dong_type.sum(axis=1)

print("📊 행정동별 상권유형 개수 (상위 10개)")
print("=" * 60)
print(dong_type.sort_values('총_상권수', ascending=False).head(10))

In [ ]:
# 행정동별 상권유형 통계
print("\n📈 행정동별 상권유형 기초 통계")
print("=" * 60)
print(dong_type.describe())

## 5. 자치구별 상권유형 분포

In [ ]:
# 자치구별 상권유형 집계
gu_type = df.groupby(['자치구_코드_명', '상권_구분_코드_명']).size().unstack(fill_value=0)
gu_type['총_상권수'] = gu_type.sum(axis=1)

print("📊 자치구별 상권유형 개수")
print("=" * 60)
print(gu_type.sort_values('총_상권수', ascending=False))

In [ ]:
# 자치구별 상권유형 시각화
gu_type_plot = gu_type.drop(columns='총_상권수').sort_values('골목상권', ascending=True)

fig, ax = plt.subplots(figsize=(12, 8))
gu_type_plot.plot(kind='barh', stacked=True, ax=ax, color=['#66b3ff', '#ff9999', '#99ff99', '#ffcc99'])
plt.title('자치구별 상권유형 분포', fontsize=14)
plt.xlabel('상권 수', fontsize=12)
plt.ylabel('자치구', fontsize=12)
plt.legend(title='상권유형', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 6. 결측치 확인

In [ ]:
print("🔍 결측치 확인")
print("=" * 50)
null_counts = df.isnull().sum()
null_pct = (null_counts / len(df) * 100).round(2)
null_df = pd.DataFrame({'결측치 수': null_counts, '비율(%)': null_pct})
null_with_values = null_df[null_df['결측치 수'] > 0]

if len(null_with_values) == 0:
    print("✅ 결측치 없음!")
else:
    print(null_with_values)

## 7. 영역 면적 분석

In [ ]:
print("📐 상권유형별 영역 면적 통계 (㎡)")
print("=" * 50)
print(df.groupby('상권_구분_코드_명')['영역_면적'].describe())

In [ ]:
# 상권유형별 영역 면적 박스플롯
plt.figure(figsize=(10, 5))
df.boxplot(column='영역_면적', by='상권_구분_코드_명', grid=False)
plt.title('상권유형별 영역 면적 분포', fontsize=14)
plt.suptitle('')  # 기본 제목 제거
plt.xlabel('상권유형', fontsize=12)
plt.ylabel('영역 면적 (㎡)', fontsize=12)
plt.tight_layout()
plt.show()

## 8. 데이터 양 적합성 판단

In [ ]:
print("🎯 데이터 양 적합성 판단")
print("=" * 50)

row_count = len(df)
type_count = df['상권_구분_코드_명'].nunique()
dong_count = df['행정동_코드'].nunique()
null_pct_type = df['상권_구분_코드_명'].isnull().sum() / len(df) * 100

# 판단
print(f"{'✅' if row_count >= 500 else '❌'} 데이터 양: {row_count:,}개 (기준: 500개 이상)")
print(f"{'✅' if type_count >= 3 else '❌'} 상권유형 수: {type_count}개 (기준: 3개 이상)")
print(f"{'✅' if dong_count >= 100 else '❌'} 행정동 수: {dong_count}개 (기준: 100개 이상)")
print(f"{'✅' if null_pct_type < 10 else '❌'} 결측치: {null_pct_type:.1f}% (기준: 10% 미만)")

## 9. 회귀분석용 데이터 변환 예시

In [ ]:
# 행정동별 상권유형 개수로 변환 (회귀분석 독립변수로 사용 가능)
dong_type_for_regression = dong_type.reset_index()

print("📊 회귀분석용 행정동별 상권유형 데이터 (상위 10개)")
print("=" * 60)
print(dong_type_for_regression.head(10))
print()
print(f"총 행정동 수: {len(dong_type_for_regression)}개")
print("→ 이 데이터를 매출 데이터와 행정동_코드로 조인하여 회귀분석 가능!")

## 📝 최종 결론

In [ ]:
print("=" * 60)
print("📝 최종 결론")
print("=" * 60)
print(f"""
[분석 대상]
- 데이터: 서울시 상권분석서비스(영역-상권)
- 지역: 서울시 {dong_count}개 행정동

[상권유형 분포]
- 골목상권: {type_counts.get('골목상권', 0):,}개 ({type_counts.get('골목상권', 0)/len(df)*100:.1f}%)
- 전통시장: {type_counts.get('전통시장', 0):,}개 ({type_counts.get('전통시장', 0)/len(df)*100:.1f}%)
- 발달상권: {type_counts.get('발달상권', 0):,}개 ({type_counts.get('발달상권', 0)/len(df)*100:.1f}%)
- 관광특구: {type_counts.get('관광특구', 0):,}개 ({type_counts.get('관광특구', 0)/len(df)*100:.1f}%)

[데이터 규모]
- 총 상권 수: {row_count:,}개
- 분석 가능 행정동: {dong_count}개

[적합성 판단]
✅ 데이터 양: 적합 ({row_count:,}개 > 500개)
✅ 상권유형: 적합 (4가지 유형 구분)
✅ 결측치: 적합 (0%)

[활용 방안]
- 행정동별 상권유형 개수를 독립변수로 사용
- 또는 더미변수로 변환하여 회귀분석 가능

→ 상권유형 데이터, 회귀분석 진행에 충분합니다! 🎉
""")